# Load a single ROI from an OME-Zarr file
To enable 3D interactivity and easy ways to interact with the dataset in a jupyter notebook, let's load a single ROI instead of the whole well.  
If you did not create your own OME-Zarr file using Fractal, you can download the output from here:
https://zenodo.org/record/7120354

In [ ]:
#pip install anndata
#pip install matplotlib

In [1]:
import napari
import zarr
import ome_zarr
from pathlib import Path
import anndata as ad
import dask.array as da
import numpy as np
from utils_roi_loading import convert_ROI_table_to_indices, load_label_roi, load_intensity_roi

In [2]:
# Define the path to your OME-Zarr file
# (either just processed via Fractal or downloaded from Zenodo)
zarr_url = Path('/Users/joel/Desktop/20220928_2x2_Zenodo/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr')

In [3]:
# Constant settings
labels_name = 'label_DAPI'
roi_table = 'FOV_ROI_table'

# Changing settings
well = 'B/03'
roi_index_of_interest = 0
channel_index = 0

In [7]:
# Look at the ROI table
roi_an = ad.read_zarr(zarr_url / f'{well}/0/tables/{roi_table}')
ro_df = roi_an.to_df()
ro_df

,x_micrometer,y_micrometer,z_micrometer,len_x_micrometer,len_y_micrometer,len_z_micrometer,x_micrometer_original,y_micrometer_original
field_id,,,,,,,,
FOV_1,-1448.300049,-1517.699951,0.0,416.0,351.0,1.0,-1448.300049,-1517.699951
FOV_2,-1032.300049,-1517.699951,0.0,416.0,351.0,1.0,-1032.300049,-1517.699951
FOV_3,-1448.300049,-1166.699951,0.0,416.0,351.0,1.0,-1448.300049,-1166.699951
FOV_4,-1032.300049,-1166.699951,0.0,416.0,351.0,1.0,-1032.300049,-1166.699951


In [4]:
label_roi, scale_label = load_label_roi(zarr_url, well, roi_index_of_interest, labels_name, level = 0)
img_roi, scale_img = load_intensity_roi(zarr_url, well, roi_index_of_interest, channel_index, level = 0)

In [5]:
viewer = napari.Viewer()
viewer.add_image(img_roi, scale=scale_img)
label_layer = viewer.add_labels(label_roi, scale=scale_label)

# Feature visualization

In [48]:
import pandas as pd
import napari
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [49]:
measurement_table = 'nuclei'
nuclei_path = zarr_url / f'{well}/{image_index}/tables/{measurement_table}'
nuclei_an = ad.read_zarr(nuclei_path)

In [50]:
feature_df = nuclei_an.to_df()
feature_df[['area', 'mean_intensity', 'standard_deviation_intensity']]

,area,mean_intensity,standard_deviation_intensity
label,,,
1,2128.0,293.573792,53.578331
2,384.0,311.109375,106.931992
3,1312.0,231.229416,47.511070
4,2608.0,264.913727,57.151825
5,704.0,387.032684,176.512741
...,...,...,...
2962,1104.0,311.662140,75.626266
2963,1104.0,296.225555,73.479431
2964,736.0,280.744568,48.305725


In [53]:
# Map the colors => using code from feature-viz plugin
feature = 'area'
feature = 'mean_intensity'
quantiles=(0.1, 0.99)
lower_contrast_limit = feature_df[feature].quantile(quantiles[0])
upper_contrast_limit = feature_df[feature].quantile(quantiles[1])
feature_df['feature_scaled'] = (
    (feature_df[feature] - lower_contrast_limit) / (upper_contrast_limit - lower_contrast_limit)
)
feature_df.loc[feature_df['feature_scaled'] < 0, 'feature_scaled'] = 0
feature_df.loc[feature_df['feature_scaled'] > 1, 'feature_scaled'] = 1
colors = plt.cm.get_cmap('viridis')(feature_df['feature_scaled'])

#colors = plt.cm.get_cmap('viridis')(feature_df[feature])

# create color dictionary
colors_simple = dict(zip(feature_df.index.astype(int), colors))

properties_array = np.zeros(feature_df.index.astype(int).max() + 1)
properties_array[feature_df.index.astype(int)] = feature_df[feature]
label_properties = {feature: np.round(properties_array, decimals=2)}
label_properties

{'mean_intensity': array([  0.  , 293.57, 311.11, ..., 280.74, 204.43, 247.74])}

In [54]:
label_layer.properties = label_properties
label_layer.color = colors_simple

In [49]:
scale_labels

[1.0, 0.65, 0.65]

In [57]:
label_roi_np.shape

(10, 1080, 1280)

In [54]:
dapi_roi_np.shape

(10, 2160, 2560)

In [62]:
viewer.camera.angles = (130.67298740487422, -52.141052759501626, -130.7850853480948)
viewer.camera.zoom = 4.602908851234012
viewer.camera.center = (4.1393577048941435, 175.59930753221713, 204.92807327371398)

In [63]:
img_sc = viewer.screenshot('napari_screenshot_3D_segmentation.png')

In [14]:
z_pos = 0
viewer.dims.set_point(0, z_pos)

# Make single plane images

In [8]:
viewer = napari.Viewer()
viewer.add_image(dapi_roi_np, scale=scale_img)

Assistant skips harvesting pyclesperanto as it's not installed.


/Users/joel/opt/miniconda3/envs/napari-assistant/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'dapi_roi_np' at 0x1278d3340>

In [22]:
viewer.add_image(dapi_roi_np, scale=scale_img)

<Image layer 'dapi_roi_np [2]' at 0x2d2f6a190>

In [23]:
for z_pos in range(10):
    viewer.dims.set_point(0, z_pos)
    img_sc = viewer.screenshot(f'singlePlanes_ROI{roi_index_of_interest}_z{z_pos}_c{ind_channel}.png')